In [1]:
from mde.data.mono_dataset import KITTIRAWDataset
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import lightning as L
from torch import optim, nn, utils, Tensor
from training_module import MDE

mde = MDE()
dataset = KITTIRAWDataset('/app/data/data_depth_annotated/2011_09_26',
                                            100,332,
                                           [0], 1, is_train=True)
val_dataset = KITTIRAWDataset('/app/data/data_depth_annotated/2011_09_26',
                                            100,332,
                                           [0], 3, is_train=True)
train_loader = utils.data.DataLoader(dataset)
val_loader = utils.data.DataLoader(val_dataset)

trainer = L.Trainer(limit_train_batches=100, max_epochs=10)
trainer.fit(model=mde, train_dataloaders=train_loader,val_dataloaders=val_loader)





/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] tensor(100.2043, grad_fn=<AddBackward0>)
tensor(100.2043, grad_fn=<AddBackward0>)
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.57it/s, v_num=166, g_loss=50.50, d_loss=100.0]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 1:   0%|          | 0/1 [00:00<?, ?it/s, v_num=166, g_loss=50.50, d_loss=100.0, val_loss=nan.0]        

RuntimeError: all elements of target should be between 0 and 1

In [ ]:
e=train_loader.dataset

In [ ]:
list(e)

In [ ]:
list([i.tolist() for i in e[('color',0,0)]])

In [ ]:
import cv2
cv2.merge(tuple([i.tolist() for i in e[('color',0,0)]]))

In [ ]:
plt.imshow(e['depth_gt'].squeeze(0))

In [ ]:
from glob import glob
import math
import random
paths = glob('/app/data/data_depth_annotated/2011_09_26/*/')
tracks = set([int(i.split('/')[-2].split('_')[-2]) for i in paths])
trackf=tracks
dataset_len=len(tracks)
train = random.choices(list(tracks), k=math.ceil((dataset_len/10)*8))
tracks = tracks-set(train)
test = random.choices(list(tracks), k=dataset_len//10)
val = tracks-set(test)


In [ ]:
dataset_len/10*7

In [ ]:
!pip3 install torchinfo

In [ ]:
import torch
from torchinfo import summary

In [ ]:
from torchvision import models
import torch.nn as nn
model = models.resnet50(pretrained=True)

In [ ]:

from mde.network.discriminator import Discriminator


summary(Discriminator(), input_size=(1, 4, 100, 332),)